In [30]:
 

class  qml(object):
    def __init__(self,data):
        self.data=data
    def  get_chunck(self,index,chain_order):
        data=self.data
        if index+1-chain_order<0:
            print("chain order must   smaller  then   sequence length")
        else:
            chunck=[]
            start=index+1-chain_order
            end=index+1
        return(data[start:end])
    def  MLH(self,index,chain_order):
        #makes   probabilty  estimate according  to  degree  of   markov chain order  with the   natural maximum 
        #likelihood estimator  like this( could be generalized)
        #Q(ement/first_previous_element,second_previous_element)=
        #card({elem,first_previous_element,second_previous_element})/card({previous_element,second_previous_element})
        #form entity  history
        chunk=self.get_chunck(index,chain_order)
        nominator=chunk
        denominator=chunk[:-1]
        data=self.data
        card_nominator=sum(data[i:i+len(nominator)]==nominator for i in range(len(data)))
        card_denominator=sum(data[i:i+len(denominator)]==denominator for i in range(len(data)))
        q=card_nominator/card_denominator
        #q is the modeled probability   of    occuring   value   at inde conditioned  my   markov chain  order
        return(q)
    def  interpolation(self,lambda_list,chain_order_list):
        """lambda  list  of lambdas like  lambda1 , ....,lambdak,  chain_order_list=[index,chain_order]"""
        #print("TO DO  BEYHAN")
       
        interpolation=[]
        for  chain in chain_order_list:
            #print(self.get_chunck(chain[0],chain[1]))
            interpolation.append(self.MLH(chain[0],chain[1]))
        if  len(lambda_list)!=chain_order_list[0][1]:
            print(" crazy   dude")
        else:
            w_average=0
            for  i in range(0,len(lambda_list)):
                w_average=w_average+(lambda_list[i]*interpolation[i])
                
                
        return [interpolation,w_average]
 
 

In [3]:
data=[1,2,3,1,5,2,3,1,1]

In [4]:
l=[1/3,1/3,1/3]
 
chain_order_list=[[4,3],[4,2],[4,1]]
q=qml(data)
q.interpolation(l,chain_order_list)


TO   FIGURE OUT  EMPIRICAL  RESULT ON  SOME     BIG  DATA SET
[3, 1, 5]
[1, 5]
[5]


[[0.5, 0.25, 0.1111111111111111], 0.28703703703703703]

In [5]:
import pandas  as pd
import numpy as np
train=pd.read_csv('train3.csv')
test=pd.read_csv('test3.csv')

In [6]:
train.loc[1]

action_type                   27
combined_shot_type             4
game_event_id                 35
game_id                 20000012
lat                           33
loc_x                       -101
loc_y                        135
lon                         -118
minutes_remaining              7
period                         1
playoffs                       0
season                         5
seconds_remaining             45
shot_distance                 16
shot_made_flag                 1
shot_type                      1
shot_zone_area                 3
shot_zone_basic                5
shot_zone_range                1
team_id               1610612747
team_name                      1
matchup                       29
opponent                      26
shot_id                        3
year                        2000
month                         10
day                           31
Name: 1, dtype: int64

In [7]:
#action type and  game event  id seems  to  be   most  correlated  features
#could  make   trigram in the following way:
#q(shot_made_flag|action_type  and game_event_id)
#ex:sentence1 from  row 1=[27,4,0]
#sentence2 from row2=[27,4,1]
#sentence3 from row3=[27,4,0 ]
# and so   on 
# on this particular  example for only 3 rows   language  is:
row1=[27,4,0]
row2=[27,4,1]
row3=[27,4,0]
data=row1+row2+row3
data
""" data  means   language   amde   by every sentence obtained from row"""
print(data)
l=[1/3,1/3,1/3]
 
chain_order_list=[[2,3],[2,2],[2,1]]
q=qml(data)
q.interpolation(l,chain_order_list)
 


[27, 4, 0, 27, 4, 1, 27, 4, 0]
TO   FIGURE OUT  EMPIRICAL  RESULT ON  SOME     BIG  DATA SET
[27, 4, 0]
[4, 0]
[0]


[[0.6666666666666666, 0.6666666666666666, 0.2222222222222222],
 0.5185185185185185]

In [8]:
2/3

0.6666666666666666

In [9]:
2/9

0.2222222222222222

In [10]:
1/3*2/3+1/3*2/3+1/3*2/9#this  0.51 is the prob of   being  1   , thus   what  we are predicting

0.5185185185185185

In [11]:
#ISSUES : 1) FOREVERY   ROW :    GET  AS LIST ELEMENTS FROM  COLUMNS  ACTION_TYPE AND  GAME_EVENT_ID ALSO SHOT_MADE_FLAG
# 2)CREATE DICTIONARY 
#3)QUERY  THING
#4)Create  predictions   for test ??????CAUTION  WHAT  HAPPENS  IF PARTICULAR  TRIGRAM NOT IN TRAIN :(

In [12]:
train.head(1)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_zone_basic,shot_zone_range,team_id,team_name,matchup,opponent,shot_id,year,month,day
0,27,4,12,20000012,34,-157,0,-118,10,1,...,5,3,1610612747,1,29,26,2,2000,10,31


In [31]:
def  create_data(df, columns,index):
    #3 d columns
    gram3=df[columns[0]][index]
    gram2=df[columns[1]][index]
    gram1=df[columns[2]][index]
    subset=df[(df[columns[0]]==gram3)&(df[columns[1]]==gram2)]
    subset=pd.DataFrame(subset,columns=columns)
    import itertools
    subset=subset[:].values.tolist()
    subset=list(itertools.chain(*subset))
    #subset now  is list  with   history  of particular   trigram
 
 
    return subset

In [32]:
s=create_data(train[0:10],columns,3)

In [33]:
s

[6, 2, 1]

In [34]:
l=[1/3,1/3,1/3]
 
chain_order_list=[[2,3],[2,2],[2,1]]
q=qml(s)
q.interpolation(l,chain_order_list)[1]

0.7777777777777777

In [ ]:
newfeature=[]
for  i in range(train.shape[0]):
    s=create_data(train,columns,i)
    l=[1/3,1/3,1/3]
    chain_order_list=[[2,3],[2,2],[2,1]]
    q=qml(s)
    interpolation_result=q.interpolation(l,chain_order_list)[1]
    newfeature.append(interpolation_result)

In [39]:
len(newfeature)

1820

In [40]:
newfeature

[0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.759619541720709,
 0.5235118295865959,
 0.29970081502114926,
 0.5235118295865959,
 0.1966909142775638,
 0.5235118295865959,
 0.5235118295865959,
 0.1966909142775638,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.1966909142775638,
 0.1966909142775638,
 0.1966909142775638,
 0.29970081502114926,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.1966909142775638,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.29970081502114926,
 0.5235118295865959,
 0.5235118295865959,
 0.29970081502114926,
 0.06375227686703096,
 0.7638057218895542,
 0.5235118295865959,
 0.29970081502114926,
 0.29970081502114926,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5235118295865959,
 0.5766448266448266,
 0.7638057218895542,
 0.5235118295865959,
 0.2997

In [41]:
1820*12

21840